In [97]:
filename = 'Instance Files/Saidman_200_NDD_Weight_0.txt'

In [98]:
def import_kidney_data(filename):
  """
  This function imports kidney exchange data from a text file.

  Args:
      filename: The path to the text file containing the data.

  Returns:
      A dictionary containing the following keys:
          num_pairs: The number of pairs in the instance.
          num_ndd: The number of non-directed donors (NDDs) in the instance.
          num_arcs: The total number of arcs in the instance.
          pairs: A list of dictionaries, where each dictionary represents a pair and contains the following keys:
              id: The ID of the pair.
              is_ndd: A boolean indicating whether the pair is an NDD (True) or a directed pair (False).
              donor_blood_type: The blood type of the donor (0 for A, 1 for B, 2 for AB, 3 for O).
              patient_blood_type: The blood type of the patient (0 for A, 1 for B, 2 for AB, 3 for O).
              patient_vpra: The vPRA score of the patient (0 for vPRA below 0.5, 1 for vPRA between 0.5 and 0.85, 2 for vPRA above 0.85).
          arcs: A list of dictionaries, where each dictionary represents an arc and contains the following keys:
              donor_id: The ID of the donor pair in the arc.
              patient_id: The ID of the patient pair in the arc.
              weight: The weight of the arc (always 1 in this case).
  """

  data = {}
  with open(filename, 'r') as f:
    # Read the header lines
    lines = f.readlines()
    num_pairs = int(lines[0].split(' ')[2])
    num_ndd = int(lines[1].split(' ')[2])
    num_arcs = int(lines[2].split(' ')[2])
    data['num_pairs'] = num_pairs
    data['num_ndd'] = num_ndd
    data['num_arcs'] = num_arcs
    num_things = num_pairs+num_ndd
    # Read the pairs and NDDs
    pairs = []
    for line in lines[3:num_things+3]:
      id, is_ndd, donor_blood_type, patient_blood_type, patient_vpra = map(int, line.strip().split(','))
      pairs.append({
          'id': id,
          'is_ndd': bool(is_ndd),
          'donor_blood_type': donor_blood_type,
          'patient_blood_type': patient_blood_type,
          'patient_vpra': patient_vpra,
      })
    data['pairs'] = pairs

    # Read the arcs
    arcs = []
    for line in lines[num_things+3:]:
      arc, weight = line.strip().split(',1,')
      #print(arc)
      #print(weight)
      donor_id, patient_id = arc.split(',')
      weight = int(weight.strip())
      donor_id = int(donor_id[1:])
      patient_id = int(patient_id[:-1])
      #print(patient_id)
      arcs.append({
          'donor_id': donor_id,
          'patient_id': patient_id,
          'weight': weight,
      })
    data['arcs'] = arcs

  return data

# Example usage
data = import_kidney_data(filename)
print(f"Number of pairs: {data['num_pairs']}")
print(f"Number of NDDs: {data['num_ndd']}")
print(f"Number of arcs: {data['num_arcs']}")



# Access information about a specific arc
arc = data['arcs'][205]
print(f"Donor ID: {arc['donor_id']}")
print(f"Patient ID: {arc['patient_id']}")
print(f"Weight: {arc['weight']}")

# Access information about a specific pair
#pair = data['pairs'][arc['donor_id']]
pair = data['pairs'][49]
print(f"Pair ID: {pair['id']}")
print(f"Is NDD: {pair['is_ndd']}")
print(f"Donor blood type: {pair['donor_blood_type']}")
print(f"Patient blood type: {pair['patient_blood_type']}")
print(f"Patient vpra: {pair['patient_vpra']}")

print(data)

Number of pairs: 190
Number of NDDs: 10
Number of arcs: 9645
Donor ID: 3
Patient ID: 154
Weight: 58
Pair ID: 49
Is NDD: False
Donor blood type: 1
Patient blood type: 2
Patient vpra: 0
{'num_pairs': 190, 'num_ndd': 10, 'num_arcs': 9645, 'pairs': [{'id': 0, 'is_ndd': False, 'donor_blood_type': 3, 'patient_blood_type': 1, 'patient_vpra': 0}, {'id': 1, 'is_ndd': False, 'donor_blood_type': 0, 'patient_blood_type': 1, 'patient_vpra': 0}, {'id': 2, 'is_ndd': False, 'donor_blood_type': 2, 'patient_blood_type': 1, 'patient_vpra': 0}, {'id': 3, 'is_ndd': False, 'donor_blood_type': 1, 'patient_blood_type': 1, 'patient_vpra': 0}, {'id': 4, 'is_ndd': False, 'donor_blood_type': 0, 'patient_blood_type': 1, 'patient_vpra': 2}, {'id': 5, 'is_ndd': False, 'donor_blood_type': 1, 'patient_blood_type': 0, 'patient_vpra': 0}, {'id': 6, 'is_ndd': False, 'donor_blood_type': 1, 'patient_blood_type': 0, 'patient_vpra': 0}, {'id': 7, 'is_ndd': False, 'donor_blood_type': 2, 'patient_blood_type': 0, 'patient_vpra'

In [99]:
import networkx as nx

In [100]:
#Find All Cycles


def create_graph(data):
    G = nx.DiGraph()
    for pair in data['pairs']:
        G.add_node(pair['id'])
    for arc in data['arcs']:
        G.add_edge(arc['donor_id'], arc['patient_id'], weight=arc['weight'])
    return G

def findPaths(G,u,n):
    if n==0:
        return [[u]]
    paths = [[u]+path for neighbor in G.neighbors(u) for path in findPaths(G,neighbor,n-1)]
    return paths

def find_cycles(G,u,n):
    paths = findPaths(G,u,n)
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x ==u for x in path) == 2]


In [101]:
G = create_graph(data)

In [103]:
k=3
c = []
p = []

def set_weight(dict_list, d, p, weight_key="weight"):

    for dct in dict_list:
        #print("donor: {}".format(dct.get("donor_id")))
        #print(dct.get(weight_key))
        if dct.get("donor_id") == d and dct.get("patient_id") == p:
            return dct.get(weight_key)
            
cycle_count = 0
path_count = 0
for l in range(1,k):
    for node in G.nodes:
        #print(type(node))
        for cyc in list(find_cycles(G, node, l)):
            cyc_sucess = 0
            cyc_weight = 0
            for n in range(1,len(cyc)):
                cyc_sucess += data['pairs'][cyc[n]]['patient_vpra']
                #print(cyc[n])
                #print(cyc[n-1])
                cyc_weight += set_weight(data['arcs'],cyc[n-1],cyc[n])
                #print('sucess')
            c.append({'id' : cycle_count, 'cycle':cyc, 'vpra_sum':cyc_sucess, 'weight_sum':cyc_weight})  
            cycle_count += 1
            
for l in range(1,k-1):
    for node in G.nodes:
        if data['pairs'][node]['is_ndd']:
            for path in list(findPaths(G, node, l)):
                path_sucess = 0
                path_weight = 0
                for n in range(1,len(path)):
                    path_sucess += data['pairs'][path[n]]['patient_vpra']
                    path_weight += set_weight(data['arcs'],path[n-1],path[n])
                p.append({'id':cycle_count,'path':path, 'vpra_sum':path_sucess, 'weight_sum':path_weight}) 
                cycle_count += 1
print(c)
print(p)

[{'id': 0, 'cycle': (1, 3, 1), 'vpra_sum': 0, 'weight_sum': 96}, {'id': 1, 'cycle': (1, 5, 1), 'vpra_sum': 0, 'weight_sum': 68}, {'id': 2, 'cycle': (1, 6, 1), 'vpra_sum': 0, 'weight_sum': 110}, {'id': 3, 'cycle': (1, 8, 1), 'vpra_sum': 0, 'weight_sum': 53}, {'id': 4, 'cycle': (1, 18, 1), 'vpra_sum': 0, 'weight_sum': 54}, {'id': 5, 'cycle': (1, 21, 1), 'vpra_sum': 0, 'weight_sum': 116}, {'id': 6, 'cycle': (1, 23, 1), 'vpra_sum': 0, 'weight_sum': 155}, {'id': 7, 'cycle': (1, 27, 1), 'vpra_sum': 0, 'weight_sum': 26}, {'id': 8, 'cycle': (1, 30, 1), 'vpra_sum': 0, 'weight_sum': 129}, {'id': 9, 'cycle': (1, 36, 1), 'vpra_sum': 0, 'weight_sum': 94}, {'id': 10, 'cycle': (1, 45, 1), 'vpra_sum': 0, 'weight_sum': 152}, {'id': 11, 'cycle': (1, 46, 1), 'vpra_sum': 0, 'weight_sum': 90}, {'id': 12, 'cycle': (1, 47, 1), 'vpra_sum': 0, 'weight_sum': 159}, {'id': 13, 'cycle': (1, 53, 1), 'vpra_sum': 0, 'weight_sum': 50}, {'id': 14, 'cycle': (1, 54, 1), 'vpra_sum': 0, 'weight_sum': 71}, {'id': 15, 'cycle

In [77]:
print(type(set_weight(data['arcs'],16,136)))

<class 'NoneType'>


In [61]:
type(data['arcs'][49]['weight'])

int